In [1]:
import random
import numpy as np
import random
import pickle
import letor_metrics
import pyximport
import sys
from tqdm import tqdm
pyximport.install()
import matplotlib


raw_data_train = np.loadtxt('/data/recnet_draft/ml100k/train_all_raw.csv', skiprows = 1, delimiter=',')
raw_data_test = np.loadtxt('/data/recnet_draft/ml100k/test_all_raw.csv', skiprows = 1, delimiter=',')
raw_data = np.concatenate((raw_data_train, raw_data_test))
from dataset_tt_static import TripletsDataset

ds = TripletsDataset(raw_data_train, raw_data_test, threshold_user=60, rnd_seed=42)
ds.train_test_split()

ds.init_cached_random()
import tensorflow as tf
import bprnn_ml100k_param_tune_all
import imp

N_USERS = int(max(raw_data[:, 0])) + 1
N_ITEMS = int(max(raw_data[:, 1])) + 1
N_EMBEDDINGS = 17

import tensorflow.contrib.slim as slim
imp.reload(bprnn_ml100k_param_tune_all)

#%%
def inner_network(user_emb, item_emb):
    joined_input = tf.concat([user_emb, item_emb], 1)
    net = slim.fully_connected(inputs=joined_input, num_outputs=32, activation_fn=tf.nn.relu)
#     net = slim.fully_connected(inputs=joined_input, num_outputs=64, activation_fn=tf.nn.relu)
#     net = slim.dro
    net = slim.fully_connected(inputs=net, num_outputs=1, activation_fn=None)
    return net

model = bprnn_ml100k_param_tune_all.BPR_NN(N_USERS, N_ITEMS, N_EMBEDDINGS, alpha=1, beta=0, alpha_reg=0.0001, inner_net=inner_network)
model.build_graph()
model.initialize_session()

losses = []
batch_size = 512
for n_batches, cur_optim in [(1000, model.trainer_3)]:
    for i in tqdm(range(n_batches)):
        batch = ds.sample_train_batch(n_samples=batch_size)
        fd = {
            model.user_ids:  batch['users'], 
            model.left_ids:  batch['left_items'],
            model.right_ids: batch['right_items'],
            model.target_y:  batch['y'],
        }
        el, nl, reg, t, _ = model.session.run(
            [model.embedding_loss, model.net_loss, model.regularization, model.target, cur_optim], 
            feed_dict=fd
        )
        losses.append((el, nl, reg, t))
        if i%500==0:
            user_norm = np.linalg.norm(model.weights_u)
            item_norm = np.linalg.norm(model.weights_i)
            print('[it {}] weight norms, users: {}, items: {}'.format(i, user_norm, item_norm))
            print('[it {}] metrics (emb_loss, net_loss, reg, target): {}'.format(i, losses[-1]))

Split users:  10%|▉         | 40/408 [00:00<00:01, 184.05it/s]

  2%|▏         | 16/1000 [00:00<01:52,  8.79it/s]

[it 0] weight norms, users: 73.19390106201172, items: 96.90826416015625
[it 0] metrics (emb_loss, net_loss, reg, target): (0.81928563, 0.69419539, 17.036283, 0.69589901)


 52%|█████▏    | 518/1000 [00:03<00:03, 144.95it/s]

[it 500] weight norms, users: 73.34114837646484, items: 97.07215118408203
[it 500] metrics (emb_loss, net_loss, reg, target): (0.69822323, 0.49626598, 17.217907, 0.49798778)


100%|██████████| 1000/1000 [00:06<00:00, 144.43it/s]


In [6]:
export_basename = '/data/recnet_draft/ml100k/vectors/'
export_pred = open(export_basename + 'pr_ml100k_10', 'w')
export_true = open(export_basename + 'gt_ml100k_10', 'w')

ndcg_vals = []
for u in tqdm(ds.data_keys, desc='Prediction', leave=True):
    # if not u in ds.test:
    if not u in ds.test or not ds.test[u]:
        continue
    response = np.zeros(len(ds.test[u]))
    fd = {
            model.user_ids:  (np.ones(len(ds.test[u]))*u).astype(np.int32), 
            model.left_ids:  np.array([i for (i, r) in ds.test[u]]).astype(np.int32),
        }
    response += model.session.run(model.embedding_left, feed_dict=fd)[:, 0]
    response += model.session.run(model.left_output, feed_dict=fd)[:, 0]

    # make relevances
    relevances = np.array([r for (i, r) in ds.test[u]])
    items = np.array([i for (i, r) in ds.test[u]])  # it's already sorted by true relevance
    itemsGroundTruth = np.array([i for (i,r) in ds.test[u] if r == 1])
    predicted_ranking = np.argsort(-response)

    # write down predictions
    export_pred.write(' '.join(map(str, [u] + list(items[predicted_ranking]))) + '\n')
    export_true.write(' '.join(map(str, [u] + list(itemsGroundTruth))) + '\n')

    # calc score
    gain = letor_metrics.ndcg_from_ranking(relevances, predicted_ranking, 10)
    ndcg_vals.append(gain)


# In[32]:

print(ndcg_vals)

Prediction: 100%|██████████| 408/408 [00:00<00:00, 2957.73it/s]

[1.0, 0.49125969208957582, 0.85123609415942747, nan, 1.0, 0.94039623903753067, 0.68120607411527379, 0.80481099920933907, 1.0, 0.65092092980713256, 1.0, 1.0, 0.69342640361727081, 0.43067655807339306, 0.65023152450395871, 0.64575457684945847, 0.63092975357145753, 1.0, 0.88545988157148736, 0.76063956823570356, 0.96746798348916929, 0.44202197604759996, 0.78873683487098878, 0.61686884000658904, 0.65092092980713256, 1.0, 0.5107574098602351, nan, 1.0, 0.98880260418802413, 0.22119341301481268, 0.61179523735681218, 1.0, 1.0, 1.0, 0.90682503796496661, 1.0, 0.93637921180104833, 0.64312114159402101, 0.7506372960520511, 1.0, 0.93268881912509716, 1.0, 0.52710649664054565, nan, 0.97149772446444627, nan, nan, 0.63092975357145753, 0.39215784798408643, 1.0, 0.39401519638101345, 0.93056877806322291, 1.0, 0.43067655807339306, 1.0, 1.0, 0.65092092980713256, 1.0, 1.0, 0.94956626441686443, nan, nan, 1.0, 1.0, 0.78854972088559572, 1.0, 1.0, 0.90682503796496661, 0.83187246372888257, 1.0, 0.29178833734520582, 1

In [7]:


ndcg_vals = []
for u in tqdm(ds.data_keys, desc='Prediction', leave=True):
    # if not u in ds.test:
    if not u in ds.test or not ds.test[u]:
        continue
    arr = np.array([i for (i,r) in ds.test[u] if r == 1])
    print(arr)

Prediction: 100%|██████████| 408/408 [00:00<00:00, 25366.89it/s]

[242 171 111 256]
[909 186 694 362 896]
[181 302  12  23 923 174 654 519 195 498  81 921 530 427 588  96 603 517
  32 492 172 509 507 191 525 474 213  50 269]
[]
[354 124 316]
[ 272  340  346  313 1016  307  298]
[ 381   70  255 1101  311  199  168  491  249  240  367  191]
[ 313  310  431  516   11   22  187  181   64  173  515  174  568  156   56
    7  176  496  318   98  209  199  186   95  194   96  435  234  429  203
  190   87   28  520  132  135  195   50  183  215   79  154    8  196  182
    9   12  202  475  238  425  143  434  588  172   69  746   70  463  736
  633  168  184  655  718  732  160  211   31  662  447  175  228  179   81
  229  185  141  423  188  436  367   10  222   91  511  778  389 1141  919
  582  959  403  111 1133  230   72  284  684  636  237  503  591  235  216
  191  651]
[298]
[272 182]
[246 993 250 268 307]
[ 616  193   47   58 1073  686  176  642  208  508   61  657  589  813   76
  132  655  789  357  647  693  183  518  631   86  484   50  228  